# NetCDF format: dam break example

This example plots the evolution of the liquid-gas interface as a liquid column collapses and flows into the rest of the domain, and it also shows the velocity magnitude as a function of interface position.

Samplers that do not have moving sample locations also do not have the "points" field in the NetCDF dataset. For static samplers, the locations are provided solely through the variable "coordinates". Otherwise, the data accesses in this example can be applied to other sampler types.

To generate the data required to replicate this example, run the simulation contained in test/test_files/dam_break_godunov with the sampling format set to "netcdf" (AMR-Wind must also be compiled with NetCDF).

In [11]:
import numpy as np
import netCDF4 as ncdf
import matplotlib.pyplot as plt

plt.rc('text',usetex=True)
plt.rc('font',family='serif')

loc_dir = "dam_break"
pp_dir = loc_dir + "/post_processing"

nt = 10 + 1
out_int = 20

In [ ]:
class SamplingFile(object):
    """Interface to Sampling NetCDF file"""

    def __init__(self, sampling_file = "sampling.nc", group_name = "s1"):
        """
        Args:
            stats_file (path): Absolute path to the NetCDF file
        """
        self.sampling_file = sampling_file
        self.sampling = ncdf.Dataset(self.sampling_file)
        self.fs = self.sampling["/"+group_name]
        self._points = self.fs.variables["points"][:,:,:]
        self._velocityx = self.fs.variables["velocityx"][:,:]
        self._velocityy = self.fs.variables["velocityy"][:,:]
        self._velocityz = self.fs.variables["velocityz"][:,:]

    @property
    def locations(self):
        return self._points

    @property
    def vmag(self):
        return np.sqrt(self._velocityx**2+self._velocityy**2+self._velocityz**2)

In [ ]:
plt.figure()
base_color = 0.8

sampl = SamplingFile(sampling_file=pp_dir+"/sampling00000.nc", \
                    group_name = "fs")

for n in range(nt):
    x_oo=sampl.locations[n,:,:]
    Vmag_oo = sampl.vmag[n,:]

    # Reorder arrays
    ind_0 = np.argsort(x_oo[:,0]) 
    x = np.zeros(np.shape(x_oo)[0])
    z = np.zeros(np.shape(x_oo)[0])
    Vmag = np.zeros(np.shape(x_oo)[0])
    flag = np.zeros(np.shape(x_oo)[0])
    for i in range(0, len(ind_0)): 
        x[i]= x_oo[ind_0[i],0]
        z[i]= x_oo[ind_0[i],2]
        Vmag[i] = Vmag_oo[ind_0[i]]
        # When interface is not present, z location is set to 0
        flag[i] = 1.0 if z[i] > 1e-8 else 0.
    # Shorten arrays (exclude points where interface not detected)
    nvalid = int(flag.sum())
    xshort = x[0:nvalid]
    zshort = z[0:nvalid]
    vshort = Vmag[0:nvalid]

    color = base_color - (base_color) * (n/nt)
    cstr = str(color)

    plt.plot(xshort,zshort,color=cstr)
    plt.scatter(xshort,zshort,c=vshort,cmap="jet",vmin=0.,vmax=2.5)

plt.ylabel(r'$z$',fontsize=16)
plt.xlabel(r'$x$',fontsize=16)
plt.colorbar()
plt.show()

# Line color gets darker with time
# Points colored with sampled velocity magnitude